# Cahier des charges

| Fonction | Cartographier la distance au vélo disponible le plus proche en temps réel |
| --- | --- |
| Objectif | Réaliser une couche raster de la distance au vélo disponible le plus proche à partir des données de l'API. |
| Contraintes | - Utiliser le package *rasterio*<br>- La couche doit couvrir toute zone située à moins de 5km d'une station Velo'V<br>- La résolution spatiale doit être de 50m |

## Résultat attendu
![Raster de distance attendu](img/distance.png "Raster de distance attendu")

# Algorithmie

Voici quelques grandes étapes pour vous aider à réaliser cet objectif. Ces étapes ne sont pas obligatoires, vous pouvez décider de procéder complètement autrement si vous le souhaitez. 

> ⚠️ La méthode présentée ici n'est pas optimale en production, mais a été choisie pour sa simplicité algorithmique

## 1. Récupérer les stations avec au moins un vélo disponible
On ouvre le tableau des stations et on filtre les stations avec au moins un vélo disponible.

## 2. Créer un raster aux bonnes dimensions et résolutions
On utilise le template_raster comme modèle pour en créer un nouveau.

## 3. Réaliser un masque des zones à plus de 5km d'une station
On réalise un buffer de 5km autour de chaque station (y compris celle qui n'ont pas de vélo disponible), puis on transforme ces buffers en un masque.

## 4. Calculer chaque pixel
Pour cette étape, on va réaliser une itération double : sur chaque ligne du raster, puis sur chaque colonne. 
Voici un exemple d'algorithme :

![Algorithme itération sur chaque pixel](img/Algo_CartoVelosProches.png "Algorithme itération sur chaque pixel")

Ci-dessous la description de ce qui est fait **pour chaque pixel du raster**.

### 4.1. Chercher la station la plus proche
On cherche la station avec au moins un vélo disponible la plus proche.

### 4.2. Mesurer la distance à cette station
On mesure la distance entre le point central du pixel et la station trouvée en 4.1.

### 4.3. Enregistrer cette distance dans la valeur du pixel
On stocke cette valeur dans le raster, à la position du pixel traité.

## 5. Ecrire le raster en sortie sur le disque
Une fois l'itération sur les pixels terminée, on peux enregistrer le raster sur le disque dur.

# Programmation

> ⚠️ La méthode présentée ici n'est pas optimale en production, mais a été choisie pour sa simplicité algorithmique

## 1. Récupérer les stations avec au moins un vélo disponible
Comme pour l'étape précédente, on cherche à charger dans une variable python le tableau des stations. *Geopandas* peut faire cela. 
Idem pour le filtrage, on le fait avec *pandas*.

Pour pouvoir utiliser la fonction nearest_points de shapely, nous aurons besoin de créer une unique géométrie de type MultiPoint contenant tous les points des stations.

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiPoint

gdf_stations = gpd.read_file('datasets/output/stations.gpkg')
filtered_gdf = gdf_stations.query('num_bikes_available >= 1')

stations_points = MultiPoint(list(filtered_gdf.geometry))
stations_points

## 2. Créer un raster aux bonnes dimensions et résolutions
On va utiliser rasterio pour créer le raster.

In [ ]:
import rasterio as rio
import numpy as np

resolution = 50
crs = rio.CRS.from_epsg(gdf_stations.crs.to_epsg())

# On créé la bounding box voulue pour notre raster
bbox = stations_points.envelope.buffer(5000).bounds

# On arrondis les valeurs à un multiple de la résolution voulue
bbox = [(coord//resolution)*resolution for coord in bbox]

# On calcule la taille (en pixels) de notre raster
bbox_size = (int((bbox[2]-bbox[0]) / resolution), # max_x - min_x
             int((bbox[3]-bbox[1]) / resolution)) # max_y - min_y

# On créé la transformation Affine (pour passer des coordonnées images aux coordonnée projetées)
transform=rio.transform.from_bounds(*bbox, width=bbox_size[0], height=bbox_size[1])

# On créé le profil minimal du raster pour que rasterio sache quoi créer
profile = {
    'driver': 'GTiff', 
    'dtype': 'float32', 
    'nodata': 255.0, 
    'width': bbox_size[0], 
    'height': bbox_size[1], 
    'count': 1, 
    'crs': crs, 
    'transform': transform,
}

# Et enfin, on créé un array numpy de la bonne taille et on l'enregistre dans un raster template rempli de zéros
data = np.zeros(bbox_size)

with rio.open('datasets/output/template_raster.tif', 'w', **profile) as template:
    template.write(data, 1)

## 3. Réaliser un masque des zones à plus de 5km d'une station
On réalise un buffer de 5km autour de chaque station (y compris celle qui n'ont pas de vélo disponible), puis on transforme ces buffers en un masque.

Voici comment faire avec rasterio.

In [ ]:
from rasterio.features import geometry_mask
from matplotlib import pyplot

# On fait un buffer de 5km autour des stations pour réaliser le masque
buffers = gdf_stations.buffer(5000)

# On réalise le masque (on fait appel aux métadonnées du template raster pour que le masque soit à la bonne taille)
mask = geometry_mask(list(buffers), template.shape, template.transform)

pyplot.imshow(mask, cmap='Blues')

In [ ]:
# Enregistrez ce masque dans un fichier


## 4. Calculer chaque pixel
On va faire itérer deux variables (```row``` et ```col```) pour qu'elle prennent successivement tous les numéros de ligne et tous les numéros de colonne.

In [ ]:
from shapely.geometry import Point
from shapely.ops import nearest_points

# On itère sur chaque ligne du raster
for row in range(profile['height']):
    # On itère sur chaque colonne du raster
    for col in range(profile['width']):
        # Pour vérifier si le pixel est masqué ou non
        if mask[row, col]: # Si le pixel est masqué
            # Pour modifier la valeur du pixel actuel sur le raster
            data[row, col] = profile['nodata']
            
        else: # Si le pixel n'est pas masqué
            # A vous de compléter pour extraire la distance


            # Pour modifier la valeur du pixel actuel sur le raster
            data[row, col] = distance

### 4.1. Chercher la station la plus proche
Pour chercher la station la plus proche du pixel actuel, il faut d'abord créer un Point *shapely* au centre du pixel

In [ ]:
pixel_point = Point(src.xy(150, 150)) # Point(src.xy(<row>, <col>))

On peux ensuite chercher le point le plus proche avec ```shapely.ops.nearest_points```. Cette fonction renvoie une paire de points stockée dans un tuple. Un des deux points correspond au pixel, l'autre à la station la plus proche.

In [ ]:
nearest = nearest_points(pixel_point, stations_points)
nearest

### 4.2. Mesurer la distance à cette station
Pour mesurer la distance entre le point central du pixel et la station trouvée en 4.1, on peux utiliser une méthode des objets ```shapely.Point```. 

### 4.3. Enregistrer cette distance dans la valeur du pixel
Voir l'exemple ci-dessus (en partie 4.) pour voir comment stocker la distance dans le raster.

## 5. Ecrire le raster en sortie sur le disque
On ouvre le raster en sortie en mode "write" (w), et on fait appel au profil enregistré précédemment pour que le raster soit aux bonnes dimensions, correctement projeté et à la bonne résolution.

La méthode dst.write prends comme arguments le tableau de données à écrire et le numéro de la bande sur laquelle écrire.

In [ ]:
with rio.open('datasets/output/distances.tif', 'w', **profile) as dst:
    dst.write(data, 1)